In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_model import FCMAE, ConvNeXtBlock1D


/usr/bin/python


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
# Set hyperparameters
num_epochs = 5
num_blocks = 5
kernel_size = 9
base_dim = 128
learning_rate = 5.493606514151052e-05
batch_size = 16

In [ ]:
train_file = os.path.join(univariate_data_path, 'merged_univariate_train.npy')
train_data = np.load(train_file, allow_pickle=True)
test_file = os.path.join(univariate_data_path, 'merged_univariate_test.npy')
test_data = np.load(test_file, allow_pickle=True)
print(train_data.shape)
print(test_data.shape)

print(train_data[0]['signal'].shape)
print(train_data[0])



In [ ]:
import numpy as np
import pandas as pd

# Function to create time windows from the signal data
def create_time_windows(data, window_length=12000, step_size=12000):
    """
    Create sliding windows of a specified length and step size from a list of signals.

    Args:
        data: list of dicts, each containing a 'signal' array of shape (seq_len, 1)
        window_length: number of time steps in each window
        step_size: number of time steps to move between windows (for overlap)

    Returns:
        np.ndarray: shape (num_windows, window_length), univariate windows
    """
    windows = []

    for entry in data:
        signal = entry['signal'].flatten()  # shape: (seq_len,)
        signal_length = len(signal)

        if signal_length < window_length:
            continue

        for start_idx in range(0, signal_length - window_length + 1, step_size):
            end_idx = start_idx + window_length
            window = signal[start_idx:end_idx]
            windows.append(window)

    return np.array(windows)  # shape: (num_windows, window_length)


X_train = create_time_windows(train_data)
X_test = create_time_windows(test_data)
print(X_train.shape)
print(X_test.shape)

In [ ]:
def mask_data(x, mask_ratio=0.5, patch_size=8):
    """
    Apply patch-based masking to a batch of univariate time series.

    Args:
        x: np.ndarray of shape (num_windows, window_length)
        mask_ratio: float, fraction of patches to mask
        patch_size: int, number of time steps in each patch

    Returns:
        np.ndarray: masked version of x with same shape
    """
    x_masked = np.copy(x)
    num_windows, window_length = x.shape
    num_patches = window_length // patch_size

    for i in range(num_windows):
        mask = np.random.rand(num_patches) < mask_ratio
        for j in range(num_patches):
            if mask[j]:
                start = j * patch_size
                end = (j + 1) * patch_size
                x_masked[i, start:end] = 0  # or np.nan if you prefer

    return x_masked

# Apply masking to train, validation, and test sets
mask_ratio = 0.75  # Adjust this to your desired masking ratio
masked_X_train = mask_data(X_train, mask_ratio)
masked_X_test = mask_data(X_test, mask_ratio)

# Check the shape of the masked datasets
print(f"Masked Training set shape: {masked_X_train.shape}")
print(f"Masked Test set shape: {masked_X_test.shape}")

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

# Define batch size
batch_size = batch_size

# Convert to tensors (still on CPU at this point)
masked_X_train_tensor = torch.tensor(masked_X_train, dtype=torch.float32).unsqueeze(1)
unmasked_X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)

masked_X_test_tensor = torch.tensor(masked_X_test, dtype=torch.float32).unsqueeze(1)
unmasked_X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

# Create TensorDatasets with both masked and unmasked tensors
train_dataset = TensorDataset(masked_X_train_tensor, unmasked_X_train_tensor)
test_dataset = TensorDataset(masked_X_test_tensor, unmasked_X_test_tensor)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import random

# Set random seeds for more reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameters
num_repeats = 10  # How many times to repeat training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_and_evaluate_once():
    model = model = FCMAE(in_channels=1, base_dim=base_dim, num_blocks=num_blocks, kernel_size=kernel_size)
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    criterion = nn.MSELoss()

    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        # --- EVALUATE ---
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for masked_input, original_input in test_loader:
                masked_input = masked_input.to(device)
                original_input = original_input.to(device)

                reconstructed = model(masked_input)
                loss = criterion(reconstructed, original_input)
                test_loss += loss.item()

            avg_test_loss = test_loss / len(test_loader)
            test_losses.append(avg_test_loss)

        # --- TRAIN ---
        model.train()
        running_loss = 0.0
        for masked_input, original_input in train_loader:
            masked_input = masked_input.to(device)
            original_input = original_input.to(device)

            reconstructed = model(masked_input)
            loss = criterion(reconstructed, original_input)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

    return train_losses, test_losses


# Store all runs
all_train_losses = []
all_test_losses = []

for run in range(num_repeats):
    print(f"\n=== Run {run+1}/{num_repeats} ===")
    train_losses, test_losses = train_and_evaluate_once()
    all_train_losses.append(train_losses)
    all_test_losses.append(test_losses)

# Convert to numpy arrays for easier computation
all_train_losses = np.array(all_train_losses)  # Shape: (num_repeats, num_epochs)
all_test_losses = np.array(all_test_losses)

# Compute mean and std
mean_train_losses = np.mean(all_train_losses, axis=0)
std_train_losses = np.std(all_train_losses, axis=0)
mean_test_losses = np.mean(all_test_losses, axis=0)
std_test_losses = np.std(all_test_losses, axis=0)

# --- Plotting ---
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, mean_train_losses, label="Mean Train Loss", color="blue", marker='o')
plt.fill_between(epochs, 
                 mean_train_losses - std_train_losses, 
                 mean_train_losses + std_train_losses, 
                 color="blue", alpha=0.2)

plt.plot(epochs, mean_test_losses, label="Mean Test Loss", color="red", marker='x')
plt.fill_between(epochs, 
                 mean_test_losses - std_test_losses, 
                 mean_test_losses + std_test_losses, 
                 color="red", alpha=0.2)

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title(f"Train and Test Loss over {num_repeats} Runs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Print summary ---
print("\nFinal Test Loss per run:")
for i, losses in enumerate(all_test_losses):
    print(f"Run {i+1}: Final Test Loss = {losses[-1]:.4f}")

print(f"\nAverage Final Test Loss: {np.mean(all_test_losses[:, -1]):.4f}")
print(f"Std of Final Test Loss: {np.std(all_test_losses[:, -1]):.4f}")
